<a href="https://colab.research.google.com/github/Wezz-git/AI-samples/blob/main/(NLP_TF_IDF)_Bag_of_Words_Spam_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The Business Problem: **

You work for a messaging company (like WhatsApp or a mobile carrier). The system handles millions of messages per second. You need a model to automatically flag messages as "Spam" or "Ham" (legitimate).

**Constraint:** You cannot use a massive Deep Learning model (like FinBERT) because it’s too slow and expensive to run on millions of messages. You need something fast, lightweight, and accurate.

This is Classical NLP using **TF-IDF** (Term Frequency-Inverse Document Frequency). Instead of trying to "understand" the meaning of words (like Deep Learning), we turn text into a mathematical spreadsheet based on word counts.

If a message contains "free", "winner", and "cash," it’s likely spam.

**TF-IDF** calculates a score for every word based on how rare or important it is.

**The Model:** Use of Naive Bayes (or Logistic Regression). These are for text classification—simple, incredibly fast, and surprisingly accurate for text.

In [9]:
# Data Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# NLP Tools
# TfidVectorizer - Turns text into numbere based o word importance
from sklearn.feature_extraction.text import TfidfVectorizer

# ML models
# MultinomialNB (Naive Bayes) - Standard for basic text Classifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# Metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

**Load & Clean:**

1- Dataset is 'messy'. It often has Encoding Issues (it's not standard text format). We need to tell pandas to read it as 'latin-1'.

2- It contains junk columns that needs dropping

In [10]:
# 1 - Load data with specific encoding
# Use encoding='latin-1', if not then the file crashes pandas

df = pd.read_csv('/content/sample_data/spam.csv', encoding='latin-1')

# 2 - Drop the 'junk' columns (Unnamed: 2,3,4)
# Only keep first 2 columns
df = df[['v1','v2']]

# 3 - Rename the columns (v1 is 'label', v2 is 'message')
df.rename(columns={'v1':'label','v2':'message'}, inplace=True)

# 4 - Check clean data
print("--- Spam Data Loaded ---")
print(df.head())

print("\n--- Class Balance ---")
print(df['label'].value_counts())

--- Spam Data Loaded ---
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

--- Class Balance ---
label
ham     4825
spam     747
Name: count, dtype: int64


Label Encoding & Splitting:

1 - Convert Labels: Turn 'ham' into 0 and 'spam' into 1.

2 - Split Data: Create your Train/Test sets.

In [11]:
from re import X
from sklearn.model_selection import train_test_split

# 1 - Convert labels to numbers manually
# Create a new column 'label_num' where ham=0, spam=1
df['label_num'] = df['label'].map({'ham':0, 'spam':1})

# 2 - Split data into
X = df['message']
y = df['label_num']  # y is always the target

# 3 - Split the data
# we'll reserve 20% for testing
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label_num'], test_size=0.2, random_state=42)

# 4 - Check the split
print(f"Training on {len(X_train)} messages")
print(f"Testing on {len(X_test)} messages")
print("\n--- Example Training Message ---")
print(f"Label: {y_train.iloc[0]}")
print(f"Message: {X_train.iloc[0]}")

Training on 4457 messages
Testing on 1115 messages

--- Example Training Message ---
Label: 0
Message: No I'm in the same boat. Still here at my moms. Check me out on yo. I'm half naked.


Use of TF-IDF (Term Frequency-Inverse Document Frequency

It gives every word a "score." Rare, spammy words get high scores. Common words get low scores.

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# 1 -  Vectorize the Text (Turn words into numbers)
print("Vectorizing text...")

# Initialize the tool
# 'stop_words - remove common useless words "the", "is", "at"
vectorizer = TfidfVectorizer(stop_words='english')

# Learn from train dats & transform it
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform test data (dont learn, just transform it)
X_test_tfidf = vectorizer.transform(X_test)

print(f"Vocabulary size: {len(vectorizer.get_feature_names_out())} unique words found.")

# 2 - train model
print("Training Naive Bayes Model..")
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)
print("Training done!")

print("Making predictions...")
predictions = model.predict(X_test_tfidf)

print("Evaluating model...")
print(classification_report(y_test, predictions, target_names=['ham (legit)','spam']))

# Get confusion matrix
print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_test, predictions))

Vectorizing text...
Vocabulary size: 7472 unique words found.
Training Naive Bayes Model..
Training done!
Making predictions...
Evaluating model...
              precision    recall  f1-score   support

 ham (legit)       0.96      1.00      0.98       965
        spam       1.00      0.75      0.86       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115


--- Confusion Matrix ---
[[965   0]
 [ 37 113]]


Precision = 1.00 (100%): This means Zero False Positives.